#### Задача 1 (3 балла). 

Разовьем тему с бойцами. Напишите игру (можно взять свой старый код в качестве базы), где игроку будет предложено выбрать класс героя: волшебник или боец. Вы должны учесть возможность добавления новых игровых классов (используйте наследование). У волшебника и бойца немного разные атрибуты (можно атрибуты сделать одинаковые в классе-родителе, но разные коэффициенты в классах-детях, на которые они домножаются: например, здоровье волшебника будет 1.0 от стандартного значения, а здоровье бойца - 1.5, а с маной наоборот). Также у них будут разные методы "нанести удар" и, если хотите, приветствия. Также у нашего героя, кем бы он ни был, должен быть рюкзак, в котором можно рыться и хранить ограниченный набор вещей (в частности, там хранятся зелья: по умолчанию пусть в начале игры каждому персонажу дается по три зелья). Наконец, нужен класс для противника: можете придумать любого монстра (тоже с возможностью добавления новых монстров, очевидно), с которым герой будет сражаться. Во время сражения неплохо предоставлять игроку выбор вида "нанести удар - выпить зелье", а сам урон от удара можно немного рандомизировать с помощью одноименного модуля. Можно еще реализовать и метод sleep для мирного времени, но во время боя он, конечно, не понадобится. 

In [5]:
from random import randint
from time import sleep

In [6]:
class Being:
    def __init__(self, name):
        self.name = name
        self.mana = 100
        self.hp = 100
        self.atk = 100
        self.defence = 100
        self.backpack = Backpack()

    def sleeping(self):
        sleep(8)
        print(f'\n{self.name} отдыхает...')
        self.hp += 30
        self.mana += 50
        print(f'{self.name} восстановил здоровье до {self.hp} и ману до {self.mana}.')
    
    def alive(self):
        return self.hp > 0   
     

class Backpack:
    def __init__(self):
        self.items = {'hp potion': 3}
        self.capacity = 3

    def usepotion(self, being):
        if self.items.get('hp potion', 0) > 0:
            being.hp += 20
            self.items['hp potion'] -= 1
            if being.hp > 100:
                being.hp = being.hp
            print(f'{being.name} выпил зелье. Здоровье - {being.hp}.')
        else:
            print(f'У {being.name} нет зелий.')        

    def showcontents(self):
        print(f'\nВ рюкзаке сейчас: {self.items}')


    def addpotion(self, item):
        if len(self.items) < self.capacity:
            self.items[item] = self.items.get(item, 0) + 1
            print(f'{item} добавлен в рюкзак')
        else:
            print('Не получается добавить зелье. Рюкзак полон.')       

  
class Wizard(Being):
    def __init__(self, name):
        super().__init__(name) 
        self.mana *= 1.5
        self.atk *= 0.5
        self.defence *= 0.5
        self.intro = f'Привет, я маг {self.name}!'

    def spells(self):
        if self.mana > 20:
            dmg = randint(10, 34)
            self.mana -= 20
            print(f'Маг накладывает заклинание на врага, минус {dmg} очков здоровья у врага')
            return dmg
        else:
            print(f'У мага не хватает маны для заклинания. Он бьёт врага посохом.')
            dmg = randint(1, 5)
            return dmg


class Warrior(Being):
    def __init__(self, name):
        super().__init__(name) 
        self.mana *= 0.5
        self.atk *= 1.5
        self.defence *= 1.5
        self.intro = f'Пора начать битву!'

    def axeatk(self):
        dmg = randint(20, 25)
        print(f'Боец атакует врага топором, минус {dmg} очков здоровья')
        return dmg   


class Goblin(Being):
    def __init__(self, name):
        super().__init__(name) 
        self.atk *= 0.5
        self.hp *= 2
        self.intro = f'Я отправлю тебя в разлагаться в аду!'

    def darkmagic(self):
        dmg = randint(10, 15)
        print(f'Гоблин применил чёрную магию, минус {dmg} очков здоровья')
        return dmg       


class Game:
    def __init__(self):
        self.monster = Goblin('Нилбог')
        self.hero = None

    def charchoice(self):
        print('Выберите класс героя:\n1 - Боец\n2 - Маг')
        choice = int(input('Ваш выбор (1/2): '))
        if choice == 1:
            name = input('Введите имя бойца: ')
            self.hero = Warrior(name)
            self.hero.intro
        elif choice == 2:
            name = input('Введите имя мага: ')
            self.hero = Wizard(name)
            self.hero.intro
        else:
            print('Такой выбор не предусмотрен. Введите заново')
            return self.charchoice()
        

    def fight(self):
        print(f'\n{self.hero.name} и гоблин {self.monster.name} начинают бой!')
        print(f'Монстр говорит: {self.monster.intro}')

        while self.hero.alive() and self.monster.alive():
            print('\nВаш ход:\n1 - Атака\n2 - Выпить зелье')
            yourmove = int(input('Выберите (1 или 2): '))
            if yourmove == 1:
                if isinstance(self.hero, Warrior):
                    self.monster.hp -= self.hero.axeatk()
                else:
                    self.monster.hp -= self.hero.spells()
            elif yourmove == 2:
                self.hero.backpack.usepotion(self.hero)
            else:
                print('Неверное действие. Пропуск хода...')

            if self.monster.alive():
                self.hero.hp -= self.monster.darkmagic()
                print(f'Здоровье {self.hero.name}: {self.hero.hp}')
                print(f'Здоровье {self.monster.name}: {self.monster.hp}')
            else:
                print('{self.monster.name} побежден!')
                break

        if self.hero.alive():
            print(f'{self.hero.name} победил!')
        else:
            print(f'{self.hero.name} убит монстром ...')


    def play(self):
        self.charchoice()
        #
        while self.hero.alive():
            print('\nВыберите действие:\n1 - Сразиться с монстром\n2 - Отдохнуть\n3 - Посмотреть, что в рюкзаке\n4 - Положить зелье в рюкзак')

            choice = int(input('Введите действие (1, 2 или 3): '))
            if choice == 1:
                self.fight()
                if not self.hero.alive():
                    print('Игра окончена.')
                    break
            elif choice == 2:
                self.hero.sleeping()
            elif choice == 3:
                self.hero.backpack.showcontents()
            elif choice == 4:
                self.hero.backpack.addpotion(input('Название зелья: '))
            else:
                print('Неверный выбор.')           




In [7]:
if __name__ == '__main__':
    game = Game()
    game.play()

Выберите класс героя:
1 - Боец
2 - Маг

Выберите действие:
1 - Сразиться с монстром
2 - Отдохнуть
3 - Посмотреть, что в рюкзаке
4 - Положить зелье в рюкзак

В рюкзаке сейчас: {'hp potion': 3}

Выберите действие:
1 - Сразиться с монстром
2 - Отдохнуть
3 - Посмотреть, что в рюкзаке
4 - Положить зелье в рюкзак

Галадриэль и гоблин Нилбог начинают бой!
Монстр говорит: Я отправлю тебя в разлагаться в аду!

Ваш ход:
1 - Атака
2 - Выпить зелье
Маг накладывает заклинание на врага, минус 23 очков здоровья у врага
Гоблин применил чёрную магию, минус 15 очков здоровья
Здоровье Галадриэль: 85
Здоровье Нилбог: 177

Ваш ход:
1 - Атака
2 - Выпить зелье
Маг накладывает заклинание на врага, минус 31 очков здоровья у врага
Гоблин применил чёрную магию, минус 12 очков здоровья
Здоровье Галадриэль: 73
Здоровье Нилбог: 146

Ваш ход:
1 - Атака
2 - Выпить зелье
Галадриэль выпил зелье. Здоровье - 93.
Гоблин применил чёрную магию, минус 10 очков здоровья
Здоровье Галадриэль: 83
Здоровье Нилбог: 146

Ваш ход:
1